In [188]:



def get_movie_with_rating(movie_id):
    """Adapted from source = https://github.com/celiao/tmdbsimple"""
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    
    # save the .info .releases dictionaries
    info = movie.info()
    
    releases = movie.releases()
    # Loop through countries in releases
    for c in releases['countries']:
        # if c['iso_3166_1'] == 'US':
        ## save a 'certification' key in info with the certification
        info['certification'] = c['certification']
        
        
    return info
    


In [189]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)


In [190]:
def read_and_fix_json(JSON_FILE):
    """Attempts to read in json file of records and fixes the final character
    to end with a ] if it errors.
    
    Args:
        JSON_FILE (str): filepath of JSON file
        
    Returns:
        DataFrame: the corrected data from the bad json file
    """
    try: 
        previous_df =  pd.read_json(JSON_FILE)
    
    ## If read_json throws an error
    except:
        
        ## manually open the json file
        with open(JSON_FILE,'r+') as f:
            ## Read in the file as a STRING
            bad_json = f.read()
            
            ## if the final character doesn't match first, select the right bracket
            first_char = bad_json[0]
            final_brackets = {'[':']', 
                           "{":"}"}
            ## Select expected final brakcet
            final_char = final_brackets[first_char]
            
            ## if the last character in file doen't match the first char, add it
            if bad_json[-1] != final_char:
                good_json = bad_json[:-1]
                good_json+=final_char
            else:
                raise Exception('ERROR is not due to mismatched final bracket.')
            
            ## Rewind to start of file and write new good_json to disk
            f.seek(0)
            f.write(good_json)
           
        ## Load the json file again now that its fixed
        previous_df =  pd.read_json(JSON_FILE)
        
    return previous_df
	
	

In [191]:
# Install tmdbsimple (only need to run once)
!pip install tmdbsimple



In [192]:
import pandas as pd
import numpy as np

# example making new folder with os
import os, time,json
import tmdbsimple as tmdb 
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)



['data (3) (1).csv',
 '.DS_Store',
 'title.ratings.tsv.gz.csv.gz',
 'title.basics.tsv.gz',
 'tmdb_api_results_2001.json',
 'title_basics.csv.gz',
 'data (5) (1).csv',
 'data (4) (1).csv',
 'title.ratings.tsv.gz',
 'title.basics.csv.gz',
 'title-akas-us-only.csv',
 'final_tmdb_data_2001.csv.gz',
 'data (6) (1).csv',
 'title_akas.csv.gz',
 'title.basics.tsv.gz.csv.gz',
 'title_ratings.csv.gz']

In [231]:
# Load the CSV file for 2001
data_2001 = pd.read_csv("/Users/corycates/Documents/GitHub/Data_Enrichment/Data/final_tmdb_data_2001.csv.gz", compression='gzip')

# Display the first few rows of the data
data_2001.head()


,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0035423,0.0,/hfeiSfWYujh6MKhtGTXyK3DD4nN.jpg,NaN,48000000.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 14, ...",NaN,11232.0,en,Kate & Leopold,...,76019048.0,118.0,"[{'english_name': 'French', 'iso_639_1': 'fr',...",Released,"If they lived in the same century, they'd be p...",Kate & Leopold,0.0,6.331,1220.0,AL
2,tt0114447,0.0,NaN,NaN,0.0,"[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...",NaN,151007.0,en,The Silent Force,...,0.0,90.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,They left him for dead... They should have fin...,The Silent Force,0.0,5.000,3.0,18
3,tt0116916,0.0,/rFpHBidSlhjflmnLu7BZilyKeQR.jpg,NaN,0.0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,73549.0,en,The Dark Mist,...,0.0,101.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,The Dark Mist,0.0,3.500,2.0,PG
4,tt0118589,0.0,/9NZAirJahVilTiDNCHLFcdkwkiy.jpg,NaN,22000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10402, 'n...",NaN,10696.0,en,Glitter,...,5271666.0,104.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,"In music she found her dream, her love, herself.",Glitter,0.0,4.442,130.0,6


In [232]:
# Filter movies with valid financial information
valid_financial_data = data_2001[(data_2001['budget'] > 0) | (data_2001['revenue'] > 0)]

# Count the number of such movies
num_movies_valid_financial_data = valid_financial_data.shape[0]
num_movies_valid_financial_data


345

In [233]:
# Filter out movies with 0's for both budget and revenue
filtered_data = data_2001[~((data_2001['budget'] == 0) & (data_2001['revenue'] == 0))]

# Count the number of movies after filtering
num_movies_after_filtering = filtered_data.shape[0]
num_movies_after_filtering


346

In [234]:
# Count the number of movies in each certification category
certification_counts = filtered_data['certification'].value_counts()
certification_counts


12        49
16        34
R         23
6         18
18        16
15        14
AL        10
U          8
PG-13      7
NR         6
PG         4
0          3
B          2
TP         2
M/12       2
G          2
M          2
7          2
UA         1
MA 15+     1
10         1
14         1
K12        1
R-18       1
R18+       1
11         1
9          1
T          1
15A        1
L          1
MA         1
Name: certification, dtype: int64

In [235]:
# Calculate the average revenue per certification category for main US categories
average_revenue_per_certification = filtered_data.groupby('certification')['revenue'].mean()[['G', 'PG', 'PG-13', 'R']]
average_revenue_per_certification


certification
G        9.317327e+06
PG       3.399228e+06
PG-13    2.488990e+07
R        2.396480e+07
Name: revenue, dtype: float64

In [236]:
# Calculate the average budget per certification category for main US categories
average_budget_per_certification = filtered_data.groupby('certification')['budget'].mean()[['G', 'PG', 'PG-13', 'R']]
average_budget_per_certification


certification
G        1.293000e+07
PG       2.500000e+07
PG-13    3.308571e+07
R        8.170220e+06
Name: budget, dtype: float64